In [14]:
begin
	using Pkg
	Pkg.activate(".")
	# Pkg.add("Revise")
	# Pkg.add("PlutoUI")
	# Pkg.add("BenchmarkTools")
	# Pkg.add("CairoMakie")
	# Pkg.add("DataFrames")
	# Pkg.add("CSV")
	# Pkg.add("CUDA")
	# Pkg.add("FoldsThreads")
	# Pkg.add(url = "https://github.com/lwbhahahaha/Losers.jl", rev = "master")
	# Pkg.add(url="https://github.com/Dale-Black/DistanceTransforms.jl")
end


  Activating project at `c:\Users\wenbl13\Desktop\Ashwin-Timing\distance-transforms`


In [15]:
using Revise
using PlutoUI
using BenchmarkTools 
using CairoMakie
using DataFrames
using CSV
using CUDA
using FoldsThreads
using DistanceTransforms
using DistanceTransforms: transform, transform!
using Losers

In [16]:
current_dir = pwd()
num_range = range(1, 510; length=51)
if has_cuda_gpu()
    ks = DistanceTransforms.get_GPU_kernels(Wenbo())
end
threads = Threads.nthreads()

GPU threads = 768.


16

# Distance Transforms
## 2D

In [17]:
sizes_2D = []
test_cases_2D = []
for n in num_range
	n = Int(round(n))
	_size = n^2
	append!(sizes_2D, _size)
	f = Bool.(rand([0, 1], n, n))
	push!(test_cases_2D, f)
end

In [18]:
maurer_mean_2D = []
maurer_std_2D = []

felzenszwalb_mean_2D = []
felzenszwalb_std_2D = []

felzenszwalb_inplace_mean_2D = []
felzenszwalb_inplace_std_2D = []

felzenszwalb_threaded_mean_2D = []
felzenszwalb_threaded_std_2D = []

felzenszwalb_threaded_mean_2D_depth = []
felzenszwalb_threaded_std_2D_depth = []

felzenszwalb_threaded_mean_2D_nonthread = []
felzenszwalb_threaded_std_2D_nonthread = []

felzenszwalb_threaded_mean_2D_worksteal = []
felzenszwalb_threaded_std_2D_worksteal = []

felzenszwalb_gpu_mean_2D = []
felzenszwalb_gpu_std_2D = []

for f in test_cases_2D
	####--- Maurer (ImageMorphology) ---####
	maurer = @benchmark transform($f, $Maurer())
	append!(maurer_mean_2D, BenchmarkTools.mean(maurer).time)
	append!(maurer_std_2D, BenchmarkTools.std(maurer).time)
	
	####--- Felzenszwalb ---####
	tfm = Felzenszwalb()
	felzenszwalb = @benchmark transform($boolean_indicator($f), $tfm)
	append!(felzenszwalb_mean_2D, BenchmarkTools.mean(felzenszwalb).time)
	append!(felzenszwalb_std_2D, BenchmarkTools.std(felzenszwalb).time)
	
	felzenszwalb = @benchmark transform!($boolean_indicator($f), $tfm)
	append!(felzenszwalb_inplace_mean_2D, BenchmarkTools.mean(felzenszwalb).time)
	append!(felzenszwalb_inplace_std_2D, BenchmarkTools.std(felzenszwalb).time)
	
	felzenszwalb = @benchmark transform!($boolean_indicator($f), $tfm, $threads)
	append!(felzenszwalb_threaded_mean_2D, BenchmarkTools.mean(felzenszwalb).time)
	append!(felzenszwalb_threaded_std_2D, BenchmarkTools.std(felzenszwalb).time)

	output, v, z = zeros(size(f)), ones(Int32, size(f)), ones(size(f) .+ 1)
	ex = DepthFirstEx()
	felzenszwalb = @benchmark transform!($boolean_indicator($f), $tfm, $ex; output=$output, v=$v, z=$z)
	append!(felzenszwalb_threaded_mean_2D_depth, BenchmarkTools.mean(felzenszwalb).time)
	append!(felzenszwalb_threaded_std_2D_depth, BenchmarkTools.std(felzenszwalb).time)

	output, v, z = zeros(size(f)), ones(Int32, size(f)), ones(size(f) .+ 1)
	ex = NonThreadedEx()
	felzenszwalb = @benchmark transform!($boolean_indicator($f), $tfm, $ex; output=$output, v=$v, z=$z)
	append!(felzenszwalb_threaded_mean_2D_nonthread, BenchmarkTools.mean(felzenszwalb).time)
	append!(felzenszwalb_threaded_std_2D_nonthread, BenchmarkTools.std(felzenszwalb).time)

	output, v, z = zeros(size(f)), ones(Int32, size(f)), ones(size(f) .+ 1)
	ex = WorkStealingEx()
	felzenszwalb = @benchmark transform!($boolean_indicator($f), $tfm, $ex; output=$output, v=$v, z=$z)
	append!(felzenszwalb_threaded_mean_2D_worksteal, BenchmarkTools.mean(felzenszwalb).time)
	append!(felzenszwalb_threaded_std_2D_worksteal, BenchmarkTools.std(felzenszwalb).time)

	if has_cuda_gpu()
		output, v, z = CUDA.zeros(size(f)), CUDA.ones(Int32, size(f)), CUDA.ones(size(f) .+ 1)
		felzenszwalb = @benchmark transform!($CuArray($boolean_indicator($f)), $tfm; output=$output, v=$v, z=$z)
		append!(felzenszwalb_gpu_mean_2D, BenchmarkTools.mean(felzenszwalb).time)
		append!(felzenszwalb_gpu_std_2D, BenchmarkTools.std(felzenszwalb).time)
	end
end

In [19]:
wenbo_inplace_mean_2D = []
wenbo_inplace_std_2D = []

wenbo_threaded_mean_2D = []
wenbo_threaded_std_2D = []

wenbo_threaded_mean_2D_depth = []
wenbo_threaded_std_2D_depth = []

wenbo_threaded_mean_2D_nonthread = []
wenbo_threaded_std_2D_nonthread = []

wenbo_threaded_mean_2D_worksteal = []
wenbo_threaded_std_2D_worksteal = []

wenbo_gpu_mean_2D = []
wenbo_gpu_std_2D = []

for f in test_cases_2D
	####--- Wenbo ---####
	tfm2 = Wenbo()
	wenbo = @benchmark transform($f, $tfm2)
	append!(wenbo_inplace_mean_2D, BenchmarkTools.mean(wenbo).time)
	append!(wenbo_inplace_std_2D, BenchmarkTools.std(wenbo).time)

	wenbo = @benchmark transform($f, $tfm2, $threads)
	append!(wenbo_threaded_mean_2D, BenchmarkTools.mean(wenbo).time)
	append!(wenbo_threaded_std_2D, BenchmarkTools.std(wenbo).time)

	ex = DepthFirstEx()
	wenbo = @benchmark transform($f, $tfm2, $ex)
	append!(wenbo_threaded_mean_2D_depth, BenchmarkTools.mean(wenbo).time)
	append!(wenbo_threaded_std_2D_depth, BenchmarkTools.std(wenbo).time)

	ex = NonThreadedEx()
	wenbo = @benchmark transform($f, $tfm2, $ex)
	append!(wenbo_threaded_mean_2D_nonthread, BenchmarkTools.mean(wenbo).time)
	append!(wenbo_threaded_std_2D_nonthread, BenchmarkTools.std(wenbo).time)

	ex = WorkStealingEx()
	wenbo = @benchmark transform($f, $tfm2, $ex)
	append!(wenbo_threaded_mean_2D_worksteal, BenchmarkTools.mean(wenbo).time)
	append!(wenbo_threaded_std_2D_worksteal, BenchmarkTools.std(wenbo).time)

	if has_cuda_gpu()
		f_cu = CuArray(f)
		wenbo = @benchmark transform($f_cu, $tfm2, $ks)
		append!(wenbo_gpu_mean_2D, BenchmarkTools.mean(wenbo).time)
		append!(wenbo_gpu_std_2D, BenchmarkTools.std(wenbo).time)
	end
end

In [20]:
path = current_dir * "/dt_2D.csv"
if has_cuda_gpu()
	df = DataFrame(
		sizes_2D = Float64.(sizes_2D),
		maurer_mean_2D = Float64.(maurer_mean_2D),
		maurer_std_2D = Float64.(maurer_std_2D),
		felzenszwalb_mean_2D = Float64.(felzenszwalb_mean_2D),
		felzenszwalb_std_2D = Float64.(felzenszwalb_std_2D),
		felzenszwalb_inplace_mean_2D = Float64.(felzenszwalb_inplace_mean_2D),
		felzenszwalb_inplace_std_2D = Float64.(felzenszwalb_inplace_std_2D),
		felzenszwalb_threaded_mean_2D = Float64.(felzenszwalb_threaded_mean_2D),
		felzenszwalb_threaded_std_2D = Float64.(felzenszwalb_threaded_std_2D),
		felzenszwalb_threaded_mean_2D_depth = Float64.(felzenszwalb_threaded_mean_2D_depth),
		felzenszwalb_threaded_std_2D_depth = Float64.(felzenszwalb_threaded_std_2D_depth),
		felzenszwalb_threaded_mean_2D_nonthread = Float64.(felzenszwalb_threaded_mean_2D_nonthread),
		felzenszwalb_threaded_std_2D_nonthread = Float64.(felzenszwalb_threaded_std_2D_nonthread),
		felzenszwalb_threaded_mean_2D_worksteal = Float64.(felzenszwalb_threaded_mean_2D_worksteal),
		felzenszwalb_threaded_std_2D_worksteal = Float64.(felzenszwalb_threaded_std_2D_worksteal),
		felzenszwalb_gpu_mean_2D = Float64.(felzenszwalb_gpu_mean_2D),
		felzenszwalb_gpu_std_2D = Float64.(felzenszwalb_gpu_std_2D),
		wenbo_inplace_mean_2D = Float64.(wenbo_inplace_mean_2D),
		wenbo_inplace_std_2D = Float64.(wenbo_inplace_std_2D),
		wenbo_threaded_mean_2D = Float64.(wenbo_threaded_mean_2D),
		wenbo_threaded_std_2D = Float64.(wenbo_threaded_std_2D),
		wenbo_threaded_mean_2D_depth = Float64.(wenbo_threaded_mean_2D_depth),
		wenbo_threaded_std_2D_depth = Float64.(wenbo_threaded_std_2D_depth),
		wenbo_threaded_mean_2D_nonthread = Float64.(wenbo_threaded_mean_2D_nonthread),
		wenbo_threaded_std_2D_nonthread = Float64.(wenbo_threaded_std_2D_nonthread),
		wenbo_threaded_mean_2D_worksteal = Float64.(wenbo_threaded_mean_2D_worksteal),
		wenbo_threaded_std_2D_worksteal = Float64.(wenbo_threaded_std_2D_worksteal),
		wenbo_gpu_mean_2D = Float64.(wenbo_gpu_mean_2D),
		wenbo_gpu_std_2D = Float64.(wenbo_gpu_std_2D)
	)
else
	df = DataFrame(
		sizes_2D = Float64.(sizes_2D),
		maurer_mean_2D = Float64.(maurer_mean_2D),
		maurer_std_2D = Float64.(maurer_std_2D),
		felzenszwalb_mean_2D = Float64.(felzenszwalb_mean_2D),
		felzenszwalb_std_2D = Float64.(felzenszwalb_std_2D),
		felzenszwalb_inplace_mean_2D = Float64.(felzenszwalb_inplace_mean_2D),
		felzenszwalb_inplace_std_2D = Float64.(felzenszwalb_inplace_std_2D),
		felzenszwalb_threaded_mean_2D = Float64.(felzenszwalb_threaded_mean_2D),
		felzenszwalb_threaded_std_2D = Float64.(felzenszwalb_threaded_std_2D),
		felzenszwalb_threaded_mean_2D_depth = Float64.(felzenszwalb_threaded_mean_2D_depth),
		felzenszwalb_threaded_std_2D_depth = Float64.(felzenszwalb_threaded_std_2D_depth),
		felzenszwalb_threaded_mean_2D_nonthread = Float64.(felzenszwalb_threaded_mean_2D_nonthread),
		felzenszwalb_threaded_std_2D_nonthread = Float64.(felzenszwalb_threaded_std_2D_nonthread),
		felzenszwalb_threaded_mean_2D_worksteal = Float64.(felzenszwalb_threaded_mean_2D_worksteal),
		felzenszwalb_threaded_std_2D_worksteal = Float64.(felzenszwalb_threaded_std_2D_worksteal),
		wenbo_inplace_mean_2D = Float64.(wenbo_inplace_mean_2D),
		wenbo_inplace_std_2D = Float64.(wenbo_inplace_std_2D),
		wenbo_threaded_mean_2D = Float64.(wenbo_threaded_mean_2D),
		wenbo_threaded_std_2D = Float64.(wenbo_threaded_std_2D),
		wenbo_threaded_mean_2D_depth = Float64.(wenbo_threaded_mean_2D_depth),
		wenbo_threaded_std_2D_depth = Float64.(wenbo_threaded_std_2D_depth),
		wenbo_threaded_mean_2D_nonthread = Float64.(wenbo_threaded_mean_2D_nonthread),
		wenbo_threaded_std_2D_nonthread = Float64.(wenbo_threaded_std_2D_nonthread),
		wenbo_threaded_mean_2D_worksteal = Float64.(wenbo_threaded_mean_2D_worksteal),
		wenbo_threaded_std_2D_worksteal = Float64.(wenbo_threaded_std_2D_worksteal)
	)
end
CSV.write(path, df)

"c:\\Users\\wenbl13\\Desktop\\Ashwin-Timing\\distance-transforms/dt_2D.csv"

## 3D

In [21]:
sizes_3D = []
test_cases_3D = []

for n in num_range
    n = Int(round(n))
    _size = n^3
    append!(sizes_3D, _size)
    push!(test_cases_3D, Bool.(rand([0, 1], n, n, n)))
end

In [22]:
maurer_mean_3D = []
maurer_std_3D = []

felzenszwalb_mean_3D = []
felzenszwalb_std_3D = []

felzenszwalb_inplace_mean_3D = []
felzenszwalb_inplace_std_3D = []

felzenszwalb_threaded_mean_3D = []
felzenszwalb_threaded_std_3D = []

felzenszwalb_threaded_mean_3D_depth = []
felzenszwalb_threaded_std_3D_depth = []

felzenszwalb_threaded_mean_3D_nonthread = []
felzenszwalb_threaded_std_3D_nonthread = []

felzenszwalb_threaded_mean_3D_worksteal = []
felzenszwalb_threaded_std_3D_worksteal = []

felzenszwalb_gpu_mean_3D = []
felzenszwalb_gpu_std_3D = []

for f in test_cases_3D
    ####--- Maurer (ImageMorphology) ---####
    maurer = @benchmark transform($f, $Maurer())
    append!(maurer_mean_3D, BenchmarkTools.mean(maurer).time)
    append!(maurer_std_3D, BenchmarkTools.std(maurer).time)

    ####--- Felzenszwalb ---####
    tfm = Felzenszwalb()
    felzenszwalb = @benchmark transform($boolean_indicator($f), $tfm)
    append!(felzenszwalb_mean_3D, BenchmarkTools.mean(felzenszwalb).time)
    append!(felzenszwalb_std_3D, BenchmarkTools.std(felzenszwalb).time)

    felzenszwalb = @benchmark transform!($boolean_indicator($f), $tfm)
    append!(felzenszwalb_inplace_mean_3D, BenchmarkTools.mean(felzenszwalb).time)
    append!(felzenszwalb_inplace_std_3D, BenchmarkTools.std(felzenszwalb).time)

    felzenszwalb = @benchmark transform!($boolean_indicator($f), $tfm, $threads)
    append!(felzenszwalb_threaded_mean_3D, BenchmarkTools.mean(felzenszwalb).time)
    append!(felzenszwalb_threaded_std_3D, BenchmarkTools.std(felzenszwalb).time)

    output, v, z = zeros(size(f)), ones(Int32, size(f)), ones(size(f) .+ 1)
    ex = DepthFirstEx()
    felzenszwalb = @benchmark transform!($boolean_indicator($f), $tfm, $ex; output=$output, v=$v, z=$z)
    append!(felzenszwalb_threaded_mean_3D_depth, BenchmarkTools.mean(felzenszwalb).time)
    append!(felzenszwalb_threaded_std_3D_depth, BenchmarkTools.std(felzenszwalb).time)

    output, v, z = zeros(size(f)), ones(Int32, size(f)), ones(size(f) .+ 1)
    ex = NonThreadedEx()
    felzenszwalb = @benchmark transform!($boolean_indicator($f), $tfm, $ex; output=$output, v=$v, z=$z)
    append!(felzenszwalb_threaded_mean_3D_nonthread, BenchmarkTools.mean(felzenszwalb).time)
    append!(felzenszwalb_threaded_std_3D_nonthread, BenchmarkTools.std(felzenszwalb).time)

    output, v, z = zeros(size(f)), ones(Int32, size(f)), ones(size(f) .+ 1)
    ex = WorkStealingEx()
    felzenszwalb = @benchmark transform!($boolean_indicator($f), $tfm, $ex; output=$output, v=$v, z=$z)
    append!(felzenszwalb_threaded_mean_3D_worksteal, BenchmarkTools.mean(felzenszwalb).time)
    append!(felzenszwalb_threaded_std_3D_worksteal, BenchmarkTools.std(felzenszwalb).time)

    if has_cuda_gpu()
        output, v, z = CUDA.zeros(size(f)), CUDA.ones(Int32, size(f)), CUDA.ones(size(f) .+ 1)
        felzenszwalb = @benchmark transform!($CuArray($boolean_indicator($f)), $tfm; output=$output, v=$v, z=$z)
        append!(felzenszwalb_gpu_mean_3D, BenchmarkTools.mean(felzenszwalb).time)
        append!(felzenszwalb_gpu_std_3D, BenchmarkTools.std(felzenszwalb).time)
    end
end

In [23]:
wenbo_inplace_mean_3D = []
wenbo_inplace_std_3D = []

wenbo_threaded_mean_3D = []
wenbo_threaded_std_3D = []

wenbo_threaded_mean_3D_depth = []
wenbo_threaded_std_3D_depth = []

wenbo_threaded_mean_3D_nonthread = []
wenbo_threaded_std_3D_nonthread = []

wenbo_threaded_mean_3D_worksteal = []
wenbo_threaded_std_3D_worksteal = []

wenbo_gpu_mean_3D = []
wenbo_gpu_std_3D = []

for f in test_cases_3D
    ####--- Wenbo ---####
    tfm2 = Wenbo()
    wenbo = @benchmark transform($f, $tfm2)
    append!(wenbo_inplace_mean_3D, BenchmarkTools.mean(wenbo).time)
    append!(wenbo_inplace_std_3D, BenchmarkTools.std(wenbo).time)

    wenbo = @benchmark transform($f, $tfm2, $threads)
    append!(wenbo_threaded_mean_3D, BenchmarkTools.mean(wenbo).time)
    append!(wenbo_threaded_std_3D, BenchmarkTools.std(wenbo).time)

    ex = DepthFirstEx()
    wenbo = @benchmark transform($f, $tfm2, $ex)
    append!(wenbo_threaded_mean_3D_depth, BenchmarkTools.mean(wenbo).time)
    append!(wenbo_threaded_std_3D_depth, BenchmarkTools.std(wenbo).time)

    ex = NonThreadedEx()
    wenbo = @benchmark transform($f, $tfm2, $ex)
    append!(wenbo_threaded_mean_3D_nonthread, BenchmarkTools.mean(wenbo).time)
    append!(wenbo_threaded_std_3D_nonthread, BenchmarkTools.std(wenbo).time)

    ex = WorkStealingEx()
    wenbo = @benchmark transform($f, $tfm2, $ex)
    append!(wenbo_threaded_mean_3D_worksteal, BenchmarkTools.mean(wenbo).time)
    append!(wenbo_threaded_std_3D_worksteal, BenchmarkTools.std(wenbo).time)

    if has_cuda_gpu()
        f_cu = CuArray(f)
        wenbo = @benchmark transform($f_cu, $tfm2, $ks)
        append!(wenbo_gpu_mean_3D, BenchmarkTools.mean(wenbo).time)
        append!(wenbo_gpu_std_3D, BenchmarkTools.std(wenbo).time)
    end
end

In [24]:
path = current_dir * "/dt_3D.csv"
if has_cuda_gpu()
    df = DataFrame(
        sizes_3D = Float64.(sizes_3D),
        maurer_mean_3D = Float64.(maurer_mean_3D),
        maurer_std_3D = Float64.(maurer_std_3D),
        felzenszwalb_mean_3D = Float64.(felzenszwalb_mean_3D),
        felzenszwalb_std_3D = Float64.(felzenszwalb_std_3D),
        felzenszwalb_inplace_mean_3D = Float64.(felzenszwalb_inplace_mean_3D),
        felzenszwalb_inplace_std_3D = Float64.(felzenszwalb_inplace_std_3D),
        felzenszwalb_threaded_mean_3D = Float64.(felzenszwalb_threaded_mean_3D),
        felzenszwalb_threaded_std_3D = Float64.(felzenszwalb_threaded_std_3D),
        felzenszwalb_threaded_mean_3D_depth = Float64.(felzenszwalb_threaded_mean_3D_depth),
        felzenszwalb_threaded_std_3D_depth = Float64.(felzenszwalb_threaded_std_3D_depth),
        felzenszwalb_threaded_mean_3D_nonthread = Float64.(felzenszwalb_threaded_mean_3D_nonthread),
        felzenszwalb_threaded_std_3D_nonthread = Float64.(felzenszwalb_threaded_std_3D_nonthread),
        felzenszwalb_threaded_mean_3D_worksteal = Float64.(felzenszwalb_threaded_mean_3D_worksteal),
        felzenszwalb_threaded_std_3D_worksteal = Float64.(felzenszwalb_threaded_std_3D_worksteal),
        felzenszwalb_gpu_mean_3D = Float64.(felzenszwalb_gpu_mean_3D),
        felzenszwalb_gpu_std_3D = Float64.(felzenszwalb_gpu_std_3D),
        wenbo_inplace_mean_3D = Float64.(wenbo_inplace_mean_3D),
        wenbo_inplace_std_3D = Float64.(wenbo_inplace_std_3D),
        wenbo_threaded_mean_3D = Float64.(wenbo_threaded_mean_3D),
        wenbo_threaded_std_3D = Float64.(wenbo_threaded_std_3D),
        wenbo_threaded_mean_3D_depth = Float64.(wenbo_threaded_mean_3D_depth),
        wenbo_threaded_std_3D_depth = Float64.(wenbo_threaded_std_3D_depth),
        wenbo_threaded_mean_3D_nonthread = Float64.(wenbo_threaded_mean_3D_nonthread),
        wenbo_threaded_std_3D_nonthread = Float64.(wenbo_threaded_std_3D_nonthread),
        wenbo_threaded_mean_3D_worksteal = Float64.(wenbo_threaded_mean_3D_worksteal),
        wenbo_threaded_std_3D_worksteal = Float64.(wenbo_threaded_std_3D_worksteal),
        wenbo_gpu_mean_3D = Float64.(wenbo_gpu_mean_3D),
        wenbo_gpu_std_3D = Float64.(wenbo_gpu_std_3D)
    )
else
    df = DataFrame(
        sizes_3D = Float64.(sizes_3D),
        maurer_mean_3D = Float64.(maurer_mean_3D),
        maurer_std_3D = Float64.(maurer_std_3D),
        felzenszwalb_mean_3D = Float64.(felzenszwalb_mean_3D),
        felzenszwalb_std_3D = Float64.(felzenszwalb_std_3D),
        felzenszwalb_inplace_mean_3D = Float64.(felzenszwalb_inplace_mean_3D),
        felzenszwalb_inplace_std_3D = Float64.(felzenszwalb_inplace_std_3D),
        felzenszwalb_threaded_mean_3D = Float64.(felzenszwalb_threaded_mean_3D),
        felzenszwalb_threaded_std_3D = Float64.(felzenszwalb_threaded_std_3D),
        felzenszwalb_threaded_mean_3D_depth = Float64.(felzenszwalb_threaded_mean_3D_depth),
        felzenszwalb_threaded_std_3D_depth = Float64.(felzenszwalb_threaded_std_3D_depth),
        felzenszwalb_threaded_mean_3D_nonthread = Float64.(felzenszwalb_threaded_mean_3D_nonthread),
        felzenszwalb_threaded_std_3D_nonthread = Float64.(felzenszwalb_threaded_std_3D_nonthread),
        felzenszwalb_threaded_mean_3D_worksteal = Float64.(felzenszwalb_threaded_mean_3D_worksteal),
        felzenszwalb_threaded_std_3D_worksteal = Float64.(felzenszwalb_threaded_std_3D_worksteal),
        wenbo_inplace_mean_3D = Float64.(wenbo_inplace_mean_3D),
        wenbo_inplace_std_3D = Float64.(wenbo_inplace_std_3D),
        wenbo_threaded_mean_3D = Float64.(wenbo_threaded_mean_3D),
        wenbo_threaded_std_3D = Float64.(wenbo_threaded_std_3D),
        wenbo_threaded_mean_3D_depth = Float64.(wenbo_threaded_mean_3D_depth),
        wenbo_threaded_std_3D_depth = Float64.(wenbo_threaded_std_3D_depth),
        wenbo_threaded_mean_3D_nonthread = Float64.(wenbo_threaded_mean_3D_nonthread),
        wenbo_threaded_std_3D_nonthread = Float64.(wenbo_threaded_std_3D_nonthread),
        wenbo_threaded_mean_3D_worksteal = Float64.(wenbo_threaded_mean_3D_worksteal),
        wenbo_threaded_std_3D_worksteal = Float64.(wenbo_threaded_std_3D_worksteal)
    )
end
CSV.write(path, df)

"c:\\Users\\wenbl13\\Desktop\\Ashwin-Timing\\distance-transforms/dt_3D.csv"

# Loss Functions
## 2D

In [25]:
dice_mean_2D = []
dice_std_2D = []

dice_mean_gpu_2D = []
dice_std_gpu_2D = []

hausdorff_mean_2D = []
hausdorff_std_2D = []

hausdorff_mean_gpu_2D = []
hausdorff_std_gpu_2D = []

sizes_loss_2D = []

for n in num_range
    n = Int(round(n))
    _size = n^2
    push!(sizes_loss_2D, _size)
    @info "n = $(n)"
    f1 = Bool.(rand([0, 1], n, n))
    f2 = Bool.(rand([0, 1], n, n))
    
    # DICE
    dice_loss = @benchmark dice($f1, $f2)
    append!(dice_mean_2D, BenchmarkTools.mean(dice_loss).time)
    append!(dice_std_2D, BenchmarkTools.std(dice_loss).time)


    
    # Hausdorff
    tfm = Wenbo()
    f1_dtm = transform(f1, tfm)
    f2_dtm = transform(f2, tfm)
    hausdorff_loss = @benchmark hausdorff($f1, $f2, $f1_dtm, $f2_dtm)
    append!(hausdorff_mean_2D, BenchmarkTools.mean(hausdorff_loss).time)
    append!(hausdorff_std_2D, BenchmarkTools.std(hausdorff_loss).time)


    # DICE GPU
    if has_cuda_gpu()
        f1_gpu = CuArray(f1)
        f2_gpu = CuArray(f2)
        dice_loss = @benchmark dice($f1, $f2)
        append!(dice_mean_gpu_2D, BenchmarkTools.mean(dice_loss).time)
        append!(dice_std_gpu_2D, BenchmarkTools.std(dice_loss).time)
    end

    # Hausdorff GPU
    if has_cuda_gpu()
        tfm = Wenbo()
        f1_dtm_gpu = transform(f1_gpu, tfm, ks)
        f2_dtm_gpu = transform(f2_gpu, tfm, ks)

        hausdorff_loss = @benchmark hausdorff($f1_gpu, $f2_gpu, $f1_dtm_gpu, $f2_dtm_gpu)
        append!(hausdorff_mean_gpu_2D, BenchmarkTools.mean(hausdorff_loss).time)
        append!(hausdorff_std_gpu_2D, BenchmarkTools.std(hausdorff_loss).time)
    end
end

┌ Info: n = 1
└ @ Main c:\Users\wenbl13\Desktop\Ashwin-Timing\distance-transforms\timings.ipynb:19
┌ Info: n = 11
└ @ Main c:\Users\wenbl13\Desktop\Ashwin-Timing\distance-transforms\timings.ipynb:19
┌ Info: n = 21
└ @ Main c:\Users\wenbl13\Desktop\Ashwin-Timing\distance-transforms\timings.ipynb:19
┌ Info: n = 32
└ @ Main c:\Users\wenbl13\Desktop\Ashwin-Timing\distance-transforms\timings.ipynb:19
┌ Info: n = 42
└ @ Main c:\Users\wenbl13\Desktop\Ashwin-Timing\distance-transforms\timings.ipynb:19
┌ Info: n = 52
└ @ Main c:\Users\wenbl13\Desktop\Ashwin-Timing\distance-transforms\timings.ipynb:19
┌ Info: n = 62
└ @ Main c:\Users\wenbl13\Desktop\Ashwin-Timing\distance-transforms\timings.ipynb:19
┌ Info: n = 72
└ @ Main c:\Users\wenbl13\Desktop\Ashwin-Timing\distance-transforms\timings.ipynb:19
┌ Info: n = 82
└ @ Main c:\Users\wenbl13\Desktop\Ashwin-Timing\distance-transforms\timings.ipynb:19
┌ Info: n = 93
└ @ Main c:\Users\wenbl13\Desktop\Ashwin-Timing\distance-transforms\timings.ipynb:19
┌

In [26]:
path = current_dir * "/loss_2D.csv"
if has_cuda_gpu()
    df = DataFrame(
        sizes_loss_2D = Float64.(sizes_loss_2D),
        dice_mean_2D = Float64.(dice_mean_2D),
        dice_std_2D = Float64.(dice_std_2D),
        dice_mean_gpu_2D = Float64.(dice_mean_gpu_2D),
        dice_std_gpu_2D = Float64.(dice_std_gpu_2D),
        hausdorff_mean_2D = Float64.(hausdorff_mean_2D),
        hausdorff_std_2D = Float64.(hausdorff_std_2D),
        hausdorff_mean_gpu_2D = Float64.(hausdorff_mean_gpu_2D),
        hausdorff_std_gpu_2D = Float64.(hausdorff_std_gpu_2D)
    )
else
    df = DataFrame(
        sizes_loss_2D = Float64.(sizes_loss_2D),
        dice_mean_2D = Float64.(dice_mean_2D),
        dice_std_2D = Float64.(dice_std_2D),
        hausdorff_mean_2D = Float64.(hausdorff_mean_2D),
        hausdorff_std_2D = Float64.(hausdorff_std_2D),
    )
end
CSV.write(path, df)

"c:\\Users\\wenbl13\\Desktop\\Ashwin-Timing\\distance-transforms/loss_2D.csv"

## 3D

In [27]:
dice_mean_3D = []
dice_std_3D = []

dice_mean_gpu_3D = []
dice_std_gpu_3D = []

hausdorff_mean_3D = []
hausdorff_std_3D = []

hausdorff_mean_gpu_3D = []
hausdorff_std_gpu_3D = []

sizes_loss_3D = []

for n in num_range
    n = Int(round(n))
    _size = n^3
    push!(sizes_loss_3D, _size)
    @info "n = $(n)"
    f1 = Bool.(rand([0, 1], n, n, n))
    f2 = Bool.(rand([0, 1], n, n, n))
    
    # DICE
    dice_loss = @benchmark dice($f1, $f2)
    append!(dice_mean_3D, BenchmarkTools.mean(dice_loss).time)
    append!(dice_std_3D, BenchmarkTools.std(dice_loss).time)


    
    # Hausdorff
    tfm = Wenbo()
    f1_dtm = transform(f1, tfm)
    f2_dtm = transform(f2, tfm)
    hausdorff_loss = @benchmark hausdorff($f1, $f2, $f1_dtm, $f2_dtm)
    append!(hausdorff_mean_3D, BenchmarkTools.mean(hausdorff_loss).time)
    append!(hausdorff_std_3D, BenchmarkTools.std(hausdorff_loss).time)


    # DICE GPU
    if has_cuda_gpu()
        f1_gpu = CuArray(f1)
        f2_gpu = CuArray(f2)
        dice_loss = @benchmark dice($f1, $f2)
        append!(dice_mean_gpu_3D, BenchmarkTools.mean(dice_loss).time)
        append!(dice_std_gpu_3D, BenchmarkTools.std(dice_loss).time)
    end

    # Hausdorff GPU
    if has_cuda_gpu()
        tfm = Wenbo()
        f1_dtm_gpu = transform(f1_gpu, tfm, ks)
        f2_dtm_gpu = transform(f2_gpu, tfm, ks)

        hausdorff_loss = @benchmark hausdorff($f1_gpu, $f2_gpu, $f1_dtm_gpu, $f2_dtm_gpu)
        append!(hausdorff_mean_gpu_3D, BenchmarkTools.mean(hausdorff_loss).time)
        append!(hausdorff_std_gpu_3D, BenchmarkTools.std(hausdorff_loss).time)
    end
    
end

┌ Info: n = 1
└ @ Main c:\Users\wenbl13\Desktop\Ashwin-Timing\distance-transforms\timings.ipynb:19
┌ Info: n = 11
└ @ Main c:\Users\wenbl13\Desktop\Ashwin-Timing\distance-transforms\timings.ipynb:19
┌ Info: n = 21
└ @ Main c:\Users\wenbl13\Desktop\Ashwin-Timing\distance-transforms\timings.ipynb:19
┌ Info: n = 32
└ @ Main c:\Users\wenbl13\Desktop\Ashwin-Timing\distance-transforms\timings.ipynb:19
┌ Info: n = 42
└ @ Main c:\Users\wenbl13\Desktop\Ashwin-Timing\distance-transforms\timings.ipynb:19
┌ Info: n = 52
└ @ Main c:\Users\wenbl13\Desktop\Ashwin-Timing\distance-transforms\timings.ipynb:19
┌ Info: n = 62
└ @ Main c:\Users\wenbl13\Desktop\Ashwin-Timing\distance-transforms\timings.ipynb:19
┌ Info: n = 72
└ @ Main c:\Users\wenbl13\Desktop\Ashwin-Timing\distance-transforms\timings.ipynb:19
┌ Info: n = 82
└ @ Main c:\Users\wenbl13\Desktop\Ashwin-Timing\distance-transforms\timings.ipynb:19
┌ Info: n = 93
└ @ Main c:\Users\wenbl13\Desktop\Ashwin-Timing\distance-transforms\timings.ipynb:19
┌

In [28]:
path = current_dir * "/loss_3D.csv"
if has_cuda_gpu()
    df = DataFrame(
        sizes_loss_3D = Float64.(sizes_loss_3D),
        dice_mean_3D = Float64.(dice_mean_3D),
        dice_std_3D = Float64.(dice_std_3D),
        dice_mean_gpu_3D = Float64.(dice_mean_gpu_3D),
        dice_std_gpu_3D = Float64.(dice_std_gpu_3D),
        hausdorff_mean_3D = Float64.(hausdorff_mean_3D),
        hausdorff_std_3D = Float64.(hausdorff_std_3D),
        hausdorff_mean_gpu_3D = Float64.(hausdorff_mean_gpu_3D),
        hausdorff_std_gpu_3D = Float64.(hausdorff_std_gpu_3D)
    )
else
    df = DataFrame(
        sizes_loss_3D = Float64.(sizes_loss_3D),
        dice_mean_3D = Float64.(dice_mean_3D),
        dice_std_3D = Float64.(dice_std_3D),
        hausdorff_mean_3D = Float64.(hausdorff_mean_3D),
        hausdorff_std_3D = Float64.(hausdorff_std_3D),
    )
end
CSV.write(path, df)

"c:\\Users\\wenbl13\\Desktop\\Ashwin-Timing\\distance-transforms/loss_3D.csv"